<a href="https://colab.research.google.com/github/prafful-kumar/Unet-Deblur/blob/main/Deblurring_U_NET(Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os, os.path, errno
import argparse
from PIL import Image
import numpy as np
from keras.models import Model
from keras.preprocessing.image import array_to_img
#from models import modelsClass
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from datetime import datetime 
import cv2
from keras import backend, optimizers

Defining Architecture of the deblur model

In [2]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Conv2DTranspose, MaxPooling2D, ZeroPadding2D
from keras import backend as keras

class Deblur(object):

    def __init__(self, img_rows = 270, img_cols = 480):

        self.img_rows = img_rows
        self.img_cols = img_cols

    def addPadding(self, layer, level): #height, width, level):
    
        w1, h1 = self.img_cols, self.img_rows
        w2, h2 = int(w1/2), int(h1/2)
        w3, h3 = int(w2/2), int(h2/2)
        w4, h4 = int(w3/2), int(h3/2)
        h = [h1,h2,h3,h4]
        w = [w1,w2,w3,w4]
        
        # Target width and height
        tw = w[level-1]
        th = h[level-1]
        
        # Source width and height
        lsize = keras.int_shape(layer)
        sh = lsize[1]
        sw = lsize[2]

        pw = (0, tw - sw)
        ph = (0, th - sh)

        layer = ZeroPadding2D(padding=(ph,pw),data_format="channels_last")(layer)
    
        return layer
        
    def Blind_deblur(self):

        input_blurred = Input((self.img_rows, self.img_cols,3))
        filter_in = 64
        filter_out = 64
        # conv = layers.Dropout(dropout)(conv)
        conv64 = Conv2D(filter_in, (3, 3), activation='relu', padding='same')(input_blurred)
        conv64 = Conv2D(filter_in, (3, 3), activation='relu', padding='same')(conv64)
        conv64 = BatchNormalization(axis=3)(conv64)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv64)

        conv128 = Conv2D(filter_in *2, (3, 3), activation='relu', padding='same')(pool1)
        conv128 = Conv2D(filter_in *2, (3, 3), activation='relu', padding='same')(conv128)
        conv128 = BatchNormalization(axis=3)(conv128)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv128)

        conv256 = Conv2D(filter_in *4, (3, 3), activation='relu', padding='same')(pool2)
        conv256 = Conv2D(filter_in *4, (3, 3), activation='relu', padding='same')(conv256)
        conv256 = BatchNormalization(axis=3)(conv256)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv256)

        conv512 = Conv2D(filter_in*8, (3, 3), activation='relu', padding='same')(pool3)
        conv512 = Conv2D(filter_in*8, (3, 3), activation='relu', padding='same')(conv512)
        conv512 = BatchNormalization(axis=3)(conv512)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv512)
        # bottleneck layer
        conv1024 = Conv2D(filter_in*16, (3, 3), activation='relu', padding='same')(pool4)
        conv1024 = Conv2D(filter_in*16, (3, 3), activation='relu', padding='same')(conv1024)

        up6 = Conv2DTranspose(filter_in*8, (2, 2), strides=(2, 2), padding='same')(conv1024)
        up6 = self.addPadding(up6,level=4)
        up6 = concatenate([up6,conv512], axis=3)
        up_conv512 = Conv2D(filter_in*8, (3, 3), activation='relu', padding='same')(up6)
        up_conv512 = Conv2D(filter_in*8, (3, 3), activation='relu', padding='same')(up_conv512)
        up_conv512 = BatchNormalization(axis=3)(up_conv512)

        up7 = Conv2DTranspose(filter_in*4, (2, 2), strides=(2, 2), padding='same')(up_conv512)
        up7 = self.addPadding(up7,level=3)
        up7 = concatenate([up7,conv256], axis=3)
        up_conv256 = Conv2D(filter_in*4, (3, 3), activation='relu', padding='same')(up7)
        up_conv256 = Conv2D(filter_in*4, (3, 3), activation='relu', padding='same')(up_conv256)
        up_conv256 = BatchNormalization(axis=3)(up_conv256)

        up8 = Conv2DTranspose(filter_in*2, (2, 2), strides=(2, 2), padding='same')(up_conv256)
        up8 = self.addPadding(up8,level=2)
        up8 = concatenate([up8,conv128], axis=3)
        up_conv128 = Conv2D(filter_in*2, (3, 3), activation='relu', padding='same')(up8)
        up_conv128 = Conv2D(filter_in*2, (3, 3), activation='relu', padding='same')(up_conv128)
        up_conv128 = BatchNormalization(axis=3)(up_conv128)

        up9 = Conv2DTranspose(filter_in, (2, 2), strides=(2, 2), padding='same')(up_conv128)
        up9 = self.addPadding(up9,level=1)
        up9 = concatenate([up9,conv64], axis=3)
        up_conv64 = Conv2D(filter_in, (3, 3), activation='relu', padding='same')(up9)
        up_conv64 = Conv2D(filter_in, (3, 3), activation='relu', padding='same')(up_conv64)
        up_conv64 = BatchNormalization(axis=3)(up_conv64)

        conv10 = Conv2D(filter_out, (1, 1), activation='tanh')(up_conv64)
        
        model = Model(inputs=input_blurred, outputs=conv10)
        
        return model


In [4]:
!cp -av "/content/drive/MyDrive/Train/Validation.tar.gz" "/content/sample_data"


'/content/drive/MyDrive/Train/Validation.tar.gz' -> '/content/sample_data/Validation.tar.gz'


In [ ]:
!tar --gunzip --extract --verbose --file="/content/sample_data/Validation.tar.gz"


In [ ]:
image_directory = "/content/Validation/sharp/sharp"
blur_directory = "/content/Validation/blurred/blurred"

width = 480
height = 270
image_dataset = [] 
blur_dataset = []  

images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #Remember enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'png'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+"/"+image_name)
        image = Image.fromarray(image)
        
        image = image.resize((width,height))
        image_dataset.append((1./255)*np.array(image))


blurs = os.listdir(blur_directory)
for i, image_name in enumerate(blurs):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(blur_directory+"/"+ image_name)
        image = Image.fromarray(image)
        
        image = image.resize((width,height))
        blur_dataset.append((1./255)*np.array(image))




In [ ]:
outpath = "output (demo)"
try:
    os.makedirs(outpath)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, blur_dataset, test_size = 0.20, random_state = 42)

#Sanity check, view few mages
import random
import numpy as np
image_number = random.randint(0, len(X_train))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(np.reshape(X_train[image_number], (height,width, 3)))
plt.subplot(122)
plt.imshow(np.reshape(y_train[image_number], (height,width,3)))
plt.show()


Loss Function

In [ ]:
from keras import backend as K

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)

Training

In [ ]:
blurred_path = inpath
blurred_names = os.listdir(blurred_path)

width = 480
height = 270
models = Deblur(height,width)
model = models.Blind_deblur()

adam = optimizers.Adam(lr=0.0000125, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = adam, loss = 'mean_squared_error',metrics=['accuracy', jacard_coef])


print(model.summary())


start3 = datetime.now() 
model_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = 32,
                    validation_data=(X_test, y_test ), 
                    shuffle=False,
                    epochs=50)
stop3 = datetime.now()

#Execution time of the model 
execution_time_model = stop3-start3
print("Model's execution time is: ", execution_time_model)

model.save('Unet_50epochs_mse.hdf5')

Testing

In [ ]:
for fname in blurred_names:
        
    print("Deblurring '%s' with DeepBlind" %(fname))
             
    blurred_img = Image.open(blurred_path + fname)
    blurred_img = blurred_img.resize((width,height),resample = 3)
    blurred_np = (1./255)*np.array(blurred_img)
    
            
    
            
    x = np.reshape(blurred_np,[1,height,width,3])
    prediction = model.predict(x, batch_size=1,verbose=0,steps=None)
    prediction = prediction[0,:,:,:]
            
    deblurred_img = array_to_img(prediction)
    deblurred_img.save(outpath+"/%s"%(fname))
            
print("DONE!")
print("--- %s seconds ---" % (time.time() - start_time))